In [1]:
import json
from typing import Union
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from process_base import X_t_proportional, X_t_stoploss

In [2]:
def get_ruin_of_instance(X_t: Union[X_t_proportional, X_t_stoploss], process_config: dict, max_time: float, seed: int):
    # update seed
    process_config.update(seed=seed)
    # make process instance
    x_t = X_t(**process_config)
    # generate process until max_time
    x_t.get_value_at_time(time=max_time)
    # get ruin probability
    for i_process in range(process_config['n'] + 1):
        for arr_time in x_t.inter_arrival_times[i_process]:
            if arr_time <= max_time:
                value = x_t.get_value_at_time(time=arr_time)
                if value[0] < 0 or value[i_process] < 0:
                    return True
    return False


def Psi(u: list, X_t: Union[X_t_proportional, X_t_stoploss], process_config: dict, max_time: float, n_simulations: int=1000, seed_offset: int=0):
    # set initial capital
    process_config.update(u=u)
    is_ruin = [False] * n_simulations
    for i_sim in range(n_simulations):
        is_ruin[i_sim] = get_ruin_of_instance(X_t, process_config, max_time, i_sim + seed_offset)
    return sum(is_ruin) / n_simulations

# Case 1: Proportional reinsurance

In [3]:
config_proportional = json.load(open("./configs/proportional/config_1.json"))
config_proportional

{'model_settings': {'n': 1,
  'c': [0.924, 0.756],
  'alpha': [0.6],
  'poisson_rate': [0.6, 1.2],
  'claims_rate': [1.5, 1.0]},
 's': [1.0, 0.5]}

In [4]:
x_t = X_t_proportional(
    u=[10, 5],
    n=config_proportional["model_settings"]["n"],
    c=config_proportional["model_settings"]["c"],
    alpha=config_proportional["model_settings"]["alpha"],
    poisson_rate=config_proportional["model_settings"]["poisson_rate"],
    claims_rate=config_proportional["model_settings"]["claims_rate"],
    seed=42
)

In [5]:
x_t.plot_process(max_time=1000)

In [6]:
%%time

tmp_config = {
    'n': config_proportional["model_settings"]["n"],
    'c': config_proportional["model_settings"]["c"],
    'alpha': config_proportional["model_settings"]["alpha"],
    'poisson_rate': config_proportional["model_settings"]["poisson_rate"],
    'claims_rate': config_proportional["model_settings"]["claims_rate"]
}

Psi(u=[10, 5], X_t=X_t_proportional, process_config=tmp_config, max_time=1000, n_simulations=1000, seed_offset=0)

CPU times: user 7.38 s, sys: 5.2 ms, total: 7.39 s
Wall time: 7.41 s


0.696

## Graphics

In [7]:
def collect_results(folder_path: str, config_name: str):
    results = []
    for child in Path(folder_path).iterdir():
        if child.is_file() and str(child)[-5:] == ".json":
            json_data = json.load(open(child))
            if json_data["config"] == config_name:
                results.append(json_data)
    return results


def transform_results_to_df(results: list):
    df = pd.DataFrame(results)
    return df


def make_plots(df: pd.DataFrame, ground_truth: float, n_simulations: int):
    df = df.loc[df["n_simulations"] == n_simulations].copy(deep=True)
    df["abs_diff"] = abs(df["left_side"] - df["right_side"])
    df["ground_truth"] = ground_truth

    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(f"Left and right side of proposition 1 [n_simulations: {n_simulations}]",
                        f"Absolute diff between left and right side [n_simulations: {n_simulations}]")
    )
    # add lines – part 1
    fig.add_traces([
        go.Scatter(x=df["max_time"], y=df["left_side"],
                   mode="lines+markers", name="left_side", line_color="blue"),
        go.Scatter(x=df["max_time"], y=df["right_side"],
                   mode = "lines+markers", name="right_side", line_color="red"),
        go.Scatter(x=df["max_time"], y=df["left_side"] + df["left_abserr"],
                   mode="lines", line_color="rgba(0,0,0,0)",
                   showlegend = False),
        go.Scatter(x=df["max_time"], y=df["left_side"] - df["left_abserr"],
                   mode="lines", line_color="rgba(0,0,0,0)",
                   fill="tonexty", fillcolor = "rgba(0, 0, 0, 0.2)",
                   showlegend = False),
        go.Scatter(x=df["max_time"], y=df["ground_truth"],
                   mode="lines", name="ground_truth", line_color="green", line={"dash": "dot", "width": 3}),
        ],
        rows=[1, 1, 1, 1, 1], cols=[1, 1, 1, 1, 1]
    )
    # add lines – part 2
    fig.add_traces([
        go.Scatter(x=df["max_time"], y=df["abs_diff"],
                   mode="lines+markers", name="abs_diff", line_color="orange"),
        ],
        rows=[1], cols=[2]
    )

    fig.update_layout(
        xaxis={"title": "max_time", "type": "log"},
        xaxis2={"title": "max_time", "type": "log"},
        yaxis={"title": "value", "range":[0.0, 1.1 * max(df["right_side"].max(), df["left_side"].max())]},
        yaxis2={"title": "value"}
    )
    fig.update_xaxes(tickmode="array", tickvals=sorted(df["max_time"].tolist()))

    return fig

### Config 1

In [8]:
left_results_list = collect_results(folder_path="./results/proportional_left", config_name="./configs/proportional/config_1.json")
left_results = transform_results_to_df(left_results_list).loc[:, ["n_simulations", "max_time", "int_value", "abserr"]].rename(columns={"int_value": "left_side", "abserr": "left_abserr"})

right_results_list = collect_results(folder_path="./results/proportional_right", config_name="./configs/proportional/config_1.json")
right_results = transform_results_to_df(right_results_list).loc[:, ["n_simulations", "max_time", "value"]].rename(columns={"value": "right_side"})

merge_results = pd.merge(left_results, right_results, on=["n_simulations", "max_time"], how="inner").sort_values(by=["n_simulations", "max_time"])

In [9]:
ground_truth_str=right_results.sort_values(by=["n_simulations", "max_time"], ascending=[False, False]).iloc[0]
print(ground_truth_str)

n_simulations      1000.000000
max_time         100000.000000
right_side            0.108501
Name: 20, dtype: float64


In [10]:
make_plots(merge_results, ground_truth=ground_truth_str["right_side"], n_simulations=50)

In [11]:
make_plots(merge_results, ground_truth=ground_truth_str["right_side"], n_simulations=100)

### Config 2

In [12]:
left_results_list = collect_results(folder_path="./results/proportional_left", config_name="./configs/proportional/config_2.json")
left_results = transform_results_to_df(left_results_list).loc[:, ["n_simulations", "max_time", "int_value", "abserr"]].rename(columns={"int_value": "left_side", "abserr": "left_abserr"})

right_results_list = collect_results(folder_path="./results/proportional_right", config_name="./configs/proportional/config_2.json")
right_results = transform_results_to_df(right_results_list).loc[:, ["n_simulations", "max_time", "value"]].rename(columns={"value": "right_side"})

merge_results = pd.merge(left_results, right_results, on=["n_simulations", "max_time"], how="inner").sort_values(by=["n_simulations", "max_time"])

In [13]:
ground_truth_str=right_results.sort_values(by=["n_simulations", "max_time"], ascending=[False, False]).iloc[0]
print(ground_truth_str)

n_simulations      1000.000000
max_time         100000.000000
right_side            0.364772
Name: 46, dtype: float64


In [14]:
make_plots(merge_results, ground_truth=ground_truth_str["right_side"], n_simulations=50)

In [15]:
make_plots(merge_results, ground_truth=ground_truth_str["right_side"], n_simulations=100)

# Case 2: Stop-loss reinsurance

In [16]:
config_stoploss = json.load(open("./configs/stoploss/config_1.json"))
config_stoploss

{'model_settings': {'n': 1,
  'c': [1.68, 1.26],
  'b': [50.0],
  'poisson_rate': [0.6, 1.2],
  'claims_rate': [1.5, 1.0]},
 's': [1.0, 0.5]}

In [17]:
x_t = X_t_stoploss(
    u=[10, 5],
    n=config_stoploss["model_settings"]["n"],
    c=config_stoploss["model_settings"]["c"],
    b=config_stoploss["model_settings"]["b"],
    poisson_rate=config_stoploss["model_settings"]["poisson_rate"],
    claims_rate=config_stoploss["model_settings"]["claims_rate"],
    seed=42
)

In [18]:
x_t.plot_process(max_time=100)

In [19]:
%%time

tmp_config = {
    'n': config_stoploss["model_settings"]["n"],
    'c': config_stoploss["model_settings"]["c"],
    'b': config_stoploss["model_settings"]["b"],
    'poisson_rate': config_stoploss["model_settings"]["poisson_rate"],
    'claims_rate': config_stoploss["model_settings"]["claims_rate"]
}

Psi(u=[10, 5], X_t=X_t_stoploss, process_config=tmp_config, max_time=1000, n_simulations=1000, seed_offset=0)

CPU times: user 9.19 s, sys: 5.62 ms, total: 9.19 s
Wall time: 9.22 s


0.497